## How to design a qubit (part 1)

The core of a quantum chip is the qubit, so it's very important to know how to design it. In this article, we’ll see the key steps to follow in designing a qubit and how to simulate it to verify that everything works correctly. The tools we'll use are Qiskit-metal and Ansys (Q3D and HFSS)

Qubit design begins with defining its characteristic parameters, such as transition frequency and anharmonicity, from which the effective capacitance and inductance can be derived. Suppose we want to design a qubit with a frequency of 5 GHz and an anharmonicity of 250MHz. We need to keep in mind two important formulas:

$$ \alpha \simeq E_c=\frac{e^2}{2hC_q}$$
$$ f=\frac{1}{2\pi \sqrt{L_jC_q}}$$
From these, we can extract:
$$ C_q=\frac{e^2}{2h\alpha}$$
$$ L_j=\frac{1}{(2\pi f)^2Cq}$$

In [10]:
import numpy as np
e = 1.60217657e-19  # electron charge
h = 6.62606957e-34  # Plank's

alpha=250e6
f=5e9

C=e**2/2/h/alpha
L=1/C/(2*np.pi*f)**2

print(C/1e-15)
print(L/1e-9)


77.4809178907237
13.076920924612372


In our case $C_q= 77.5$ fF, $L_j=13$ nH.

The second step consists in choosing the type of qubit we want to design (Transmon, Crossmon, etc.). Due to its simplicity, we chose the Crossmon as our first example. Later, we will see that designing a Transmon is a bit more challenging due to its geometry.

We now use Qiskit Metal, an open-source Python framework that allows us to create, visualize, and simulate superconducting quantum devices. Before we can create our qubit, we need to initialize the Qiskit Metal environment, import the required libraries and open the GUI

In [5]:
#This cell is a prerequisite, you must inserted it every time you create a new code

%load_ext autoreload  
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI,Dict, open_docs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
design = designs.DesignPlanar() 
gui = MetalGUI(design) #to open gui


In [3]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
design.overwrite_enabled = True

design_options=dict(
    cross_width= '20um',
    cross_length='145um', 
    cross_gap='20um',
    hfss_inductance='13nH',
    pos_x='2mm',
    )

q1 = TransmonCross(design, 'Q1', options=design_options)

gui.rebuild()
gui.autoscale()

After running the code, the Crossmon will appear in the GUI. It can be modified either through the code or directly within the GUI. To verify whether our qubit has the correct dimensions in terms of capacitance, we need to extract the associated capacitance matrix using Ansys Q3D.

In [4]:
from qiskit_metal.analyses.quantization import LOManalysis
c1=LOManalysis(design, "q3d")
c1.sim.run(components=['Q1'], open_terminations=[])
c1.sim.capacitance_matrix

INFO 09:50PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:50PM [load_ansys_project]: 	Opened Ansys App
INFO 09:50PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 09:50PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/simona.zaccaria4/Documents/Ansoft/
	Project:   Project31
INFO 09:50PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:50PM [connect]: 	Connected to project "Project31" and design "Design_q3d" 😀 

INFO 09:50PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:50PM [analyze]: Analyzing setup Setup
INFO 09:51PM [get_matrix]: Exporting matrix data to (C:\Users\SIMONA~1.ZAC\AppData\Local\Temp\tmpc6n2ipbc.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 500

,cross_Q1,ground_main_plane
cross_Q1,77.96968,-76.77556
ground_main_plane,-76.77556,122.01138


A matrix like the one shown aboveshould appear. As we can see, the capacitance of the Crossmon with respect to ground is 76.8 fF, which is approximately the target value we aimed to achieve. Note that, in general, you are unlikely to obtain the desired value on the first attempt. Instead, you should iterate the process—adjusting the Crossmon dimensions and rerunning the Q3D simulation—until the correct size is found.

First of all, we need to set up the Ansys HFSS simulation by specifying the number of modes, the inductance value, and a few other parameters, as shown below.

In [6]:
import pyEPR as epr
from qiskit_metal.analyses.quantization import EPRanalysis
eig_1qb = EPRanalysis(design, "hfss")

eig_1qb.sim.setup.max_passes = 12
eig_1qb.sim.setup.max_delta_f = 0.1
eig_1qb.sim.setup.min_freq_ghz=1
eig_1qb.sim.setup.n_modes = 1
eig_1qb.sim.setup.vars = Dict(Lj= '13 nH', Cj= '0 fF') 

eig_1qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 1,
 'max_delta_f': 0.1,
 'max_passes': 12,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '13 nH', 'Cj': '0 fF'}}

In [7]:
eig_1qb.sim.run(components=['Q1'], open_terminations=[],port_list=[])

INFO 12:13AM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:13AM [load_ansys_project]: 	Opened Ansys App
INFO 12:13AM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 12:13AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/simona.zaccaria4/Documents/Ansoft/
	Project:   Project31
INFO 12:13AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 12:13AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:13AM [connect]: 	Connected to project "Project31" and design "Design_q3d" 😀 

INFO 12:13AM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
INFO 12:13AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 12:13AM [analyze]: Analyzing setup Setup
12:14AM 20s INFO [get_f_convergence]: Saved convergences to c:\Users\simona.zaccaria4\Desktop\hfss_eig_f_convergence.csv


In [12]:
import numpy as np
f=1/np.sqrt(13e-9*78e-15)/2/np.pi
print(f/1e9)

4.998056418164591


The expected frequency is approximately 5 GHz, and the simulation returns a value of 4.86 GHz, which is quite close. The small discrepancy is mainly due to the capacitance not being exactly 77.5 fF and to the limited number of simulation passes.

This concludes the first part of how to design a qubit. We have seen how to calculate the capacitance and inductance values starting from the constraints on the qubit's characteristic parameters, and how to size a Crossmon accordingly to match the target values